In [1]:
import pandas as pd
import numpy as np

In [10]:
notifs = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/Notificaciones TB/GTM - TB all notifications 2012-Sep2018.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,8,9,11,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,68,69,70,72,74,78,79,80,81,83,84,85,87,89,90,91,93,94,98,102) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
casesNotif = notifs[notifs.Year<2018].assign(municipality = lambda x: x.COD_MUNI).assign(date=lambda x: x.Year.astype(int))\
.assign(department = lambda x: np.floor(x.COD_MUNI/100) ).groupby(["date", "department", "municipality"]).apply(len)

In [27]:
casesNotif = casesNotif.rename("Cases Notified_value")

In [28]:
extraM2017 = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/Extramuros2017.csv", index_col=0)

In [35]:
extraM2017[extraM2017.Year.notna()].assign(municipality = lambda x: x.MuniCode.astype(int)).assign(date = lambda x: x.Year.astype(int)).\
assign(department = lambda x: x.DeptoCode.astype(int)).groupby(["date", "department", "municipality"])["bacteriologicos.extramuros"].sum()

date  department  municipality
2017  1           101             12.0
                  108              9.0
                  109              1.0
                  110              1.0
                  112              1.0
                  114              1.0
                  115              9.0
      5           501              4.0
                  502             23.0
                  505              2.0
                  506              8.0
                  509              4.0
      9           920             10.0
      10          1001             5.0
                  1007             1.0
                  1010             6.0
                  1013             3.0
      11          1101             8.0
                  1106             2.0
                  1109             5.0
      12          1213             1.0
                  1215             4.0
                  1217             3.0
                  1222             2.0
      18          1801           